In [13]:
# Importando libraries

import selenium
from selenium import webdriver
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager

import requests
import os
from pathlib import Path
import glob

import pandas as pd
import time
import xlrd


In [14]:
def get_file_dataframe():
    
    list_of_files = glob.glob(f'{Path.home()}\Downloads\*.xls')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    filename = latest_file

    
    try:
        #print('aqui')
        workbook =  xlrd.open_workbook_xls(filename, ignore_workbook_corruption=True)
        df = pd.read_excel(workbook,sheet_name = 0)
        #print('dentro do try')
        return df
    except:
        return print('DEU RUIM')
    
def delete_files_used(times):
    
    for i in range(times):
        
        list_of_files = glob.glob(f'{Path.home()}/Downloads/*.xls')
        latest_file = max(list_of_files, key=os.path.getctime)
        
        os.remove(latest_file)
        
    return True;

def download_wait(path_to_downloads):
    
    seconds = 0
    dl_wait = True
    
    while dl_wait and seconds < 20:
        
        time.sleep(1)
        dl_wait = False
        
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.crdownload'):
                dl_wait = True
                
        seconds += 1
        
    return seconds

In [15]:
frango_xpath_list = [
    '//*[@id="imagenet-content"]/div[2]/div[2]/div[1]/div[3]/a[4]',
    '//*[@id="imagenet-content"]/div[2]/div[4]/div[3]/a[4]'
]




In [16]:
dados_dict = {
    'frango' : frango_xpath_list
}


In [17]:
frango_code_list   = ['congelado','resfriado']

In [18]:
encoder_dict =  {
    'frango': frango_code_list  
}

In [19]:
# setup-selenium


driver = webdriver.Edge(executable_path = 'C:\\Users\\Downloads\\edgedriver_win64\\msedgedriver.exe');
# Check if the current version of chromedriver exists
# and if it doesn't exist, download it automatically,
# then add chromedriver to path
driver.maximize_window()

C:\Users\guilh\AppData\Local\Temp\ipykernel_15052\4013497868.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path = 'C:\\Users\\Downloads\\edgedriver_win64\\msedgedriver.exe');


In [20]:
url = 'https://www.cepea.esalq.usp.br/br/indicador/frango.aspx'
driver.get(url)

data_dict = {}


In [21]:
k = 0
for keys, values in encoder_dict.items():
    #print(k)
    lista_xpath = dados_dict[keys]
    #url = lista_xpath[0]
    #print(keys)
    #driver.find_element('xpath',url).click()
    
    list_df = lista_xpath
    i = 0
    k = k+1
    for df_xpath,encoder in zip(list_df,values):
        #print(encoder)
        #print(i)
        name = keys+'-'+encoder
        name = name.replace('_','-') + '_brl_close_d1.json'
        
        driver.find_element('xpath',df_xpath).click()
        download_wait(f'{Path.home()}/Downloads/')
        df = get_file_dataframe()
        data_dict[name] = df
        delete_files_used(1)
        i = i +1
    
    
    
    

In [22]:
for values in data_dict.values():
    values.drop(index = values.index[0:3],axis = 0,inplace = True)
    values.drop(columns = values.columns[-1],axis =1,inplace = True)
    values.rename(columns = {values.columns[0]:'ano',values.columns[-1]:'valor-brl'},inplace = True)

In [23]:
base_path = 'C:\\Users\\guilh\\TM\\var_comodity\\JSONS\\CEPEA\\'
for name, df in data_dict.items():
    full_path = base_path+name
    if not df.empty:
    #    print(df.empty)
        df.to_json(full_path,indent =1)
    else:
        pass

In [24]:
driver.close()